 ## **[파이썬 코딩을 통한 텍스트 데이터 수집방법]**
 
 
 ### 1. BS(Beautiful Soup)을 활용한 데이터 수집 ( 네이버 블로그 )  
 
 
 ### 2. selenium을 활용한 데이터 수집 (네이버 쇼핑후기) 
 
 
 
 
   

# 1. BS를 활용한 데이터 수집 (네이버 블로그)

## 1_1. 필요한 라이브러리 설치

In [1]:
# !pip install로 PC에 일단 한번 설치한 이후에는 다시 설치할 필요없음. import는 매번 


# !pip install pandas   # 크롤링한 데이터를 쉽게 데이터프레임 형식으로 처리해주는 라이브러리
# !pip install xlsxwriter    # 크롤링한 데이터를 엑셀 파일로 저장해주는 라이브러리
# !pip install BS4    # 크롤링하는 BeautifulSoup 라이브러리
# !pip install wordcloud    # 워드클라우드를 수행해주는 라이브러리


# import re  # 정규표현식 사용하기 위한 라이브러리
# import json  # 네이버 API를 통해 가져온 JSON 파일을 처리
# import math  # 수리적 작업 처리
# import requests  # http 통신에 필요
# import urllib.request  # 웹과 관련된 내용을 편리하게 처리하기 위한 여러 라이브러리 urllib
# import urllib.error
# import urllib.parse
# from bs4 import BeautifulSoup  # 크롤링을 위한 라이브러리 
# import pandas as pd
# import xlsxwriter
# from tqdm import tqdm  # 소요시간 확인용 라이브러리


In [1]:
!pip install pandas   
!pip install xlsxwriter    
!pip install BS4   
!pip install wordcloud    

  Created wheel for BS4: filename=bs4-0.0.1-py3-none-any.whl size=1277 sha256=3add0a9095fbb267dd537e2957ea8eec8c7699e769260108d06958d06120fe13
  Stored in directory: c:\users\smell\appdata\local\pip\cache\wheels\75\78\21\68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built BS4


In [5]:
import re  
import json  
import math  
import requests  
import urllib.request 
import urllib.error
import urllib.parse
from bs4 import BeautifulSoup   
import pandas as pd
import xlsxwriter
from tqdm import tqdm  

## 1_2. 네이버 개발자 사이트에서 ID와 Secret 얻기
네이버에서 제공하는 API를 이용하려면 네이버 개발자 사이트 https://developers.naver.com 에서 어플리케이션 등록 후, Client ID는 naver_clinet_id에 넣고, Client Secret은 naver_client_secret에 넣는다.

별도의 id와 비밀번호를 받을 필요없이 아래의 ID와 비밀번호를 그대로 사용하면 된다.

In [1]:
naver_client_id = "MDzHmvlSE3_r3grDXrJG"
naver_client_secret = "hM9spkwXrZ"

## 1_3. 네이버 블로그 갯수 가져오기


In [2]:
def get_blog_count(query, display): #검색어 해당하는 블로그 전체 개수 가져오기
    encode_query = urllib.parse.quote(query)
    search_url = "https://openapi.naver.com/v1/search/blog?query=" + encode_query
    request = urllib.request.Request(search_url) 

    request.add_header("X-Naver-Client-Id", naver_client_id)  #헤더추가
    request.add_header("X-Naver-Client-Secret", naver_client_secret)

    response = urllib.request.urlopen(request) #실제 사이트에 해당하는 결과 요청
    response_code = response.getcode()  #반환되는 코드 확인

    if response_code is 200: # 서버가 요청을 제대로 처리한 경우
        response_body = response.read()
        response_body_dict = json.loads(response_body.decode('utf-8'))  # 네이버가 제공하는 JSON파일로 읽기

        print("Last build date: " + str(response_body_dict['lastBuildDate']))  # JSON 출력 중 해당 항목의 필드를 가져와 출력
        print("Total: " + str(response_body_dict['total']))
        print("Start: " + str(response_body_dict['start']))
        print("Display: " + str(response_body_dict['display']))

        if response_body_dict['total'] == 0:
            blog_count = 0
        else:
            blog_total = math.ceil(response_body_dict['total'] / int(display))

            if blog_total >= 1000:   # 검색결과를 최대 1000개까지 출력
                blog_count = 1000
            else:
                blog_count = blog_total

            print("Blog total: " + str(blog_total))
            print("Blog count: " + str(blog_count))

        return blog_count

<>:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-2-4e5a34915097>:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if response_code is 200: # 서버가 요청을 제대로 처리한 경우


## 1_4. 네이버 블로그 포스트 가져오기

In [3]:
def get_blog_post(query, display, start_index, sort):  # 검색된 블로그 내용 가져오기
    global no, df
    
    encode_query = urllib.parse.quote(query)  # 검색API에 요청할 변수로 query외에 display, start, sort 추가
    search_url = "https://openapi.naver.com/v1/search/blog?query=" + \
                encode_query + "&display=" + str(display) + "&start=" + str(start_index) + "&sort=" + sort

    request = urllib.request.Request(search_url)

    request.add_header("X-Naver-Client-Id", naver_client_id)  # 아이디와 비밀번호를 헤더에 추가
    request.add_header("X-Naver-Client-Secret", naver_client_secret)

    response = urllib.request.urlopen(request)
    response_code = response.getcode()

    if response_code is 200:
        response_body = response.read()
        response_body_dict = json.loads(response_body.decode('utf-8'))
        for item_index in range(0, len(response_body_dict['items'])):
            try:                              # item 필드 안에 있는 title 등의 필드에 대한 내용 가져오기
                remove_html_tag = re.compile('<.*?>') #HTML 태그 제거를 위한 정규표현식 함수 사용
                title = re.sub(remove_html_tag, '', response_body_dict['items'][item_index]['title'])
                link = response_body_dict['items'][item_index]['link'].replace("amp;", "")
                description = re.sub(remove_html_tag, '', response_body_dict['items'][item_index]['description'])
                blogger_name = response_body_dict['items'][item_index]['bloggername']
                blogger_link = response_body_dict['items'][item_index]['bloggerlink']
                post_date = response_body_dict['items'][item_index]['postdate']

                no += 1
                post_code = requests.get(link)
                post_text = post_code.text
                post_soup = BeautifulSoup(post_text, 'lxml')

                blog_post_content_text = ""
                for mainFrame in post_soup.select('iframe#mainFrame'):
                    blog_post_url = "http://blog.naver.com" + mainFrame.get('src')
                    blog_post_code = requests.get(blog_post_url)
                    blog_post_text = blog_post_code.text
                    blog_post_soup = BeautifulSoup(blog_post_text, 'lxml')
                    
                    for blog_post_content in blog_post_soup.find_all('div', class_='se-viewer'):
                        blog_post_content_text = blog_post_content.get_text()
                        
                    for blog_post_content in blog_post_soup.find_all('div', class_='se_doc_viewer'):
                        blog_post_content_text = blog_post_content.get_text()
 
                    for blog_post_content in blog_post_soup.select('div#postViewArea'):
                        blog_post_content_text = blog_post_content.get_text()

                df.loc[no] = [title, link, description, blogger_name, blogger_link, post_date, blog_post_content_text]
                print("#", end='')
                
            except:
                item_index += 1

<>:16: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:16: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-3-d08756aab160>:16: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if response_code is 200:


## 1_5. 네이버 블로그 텍스트 수집 시작
- 약 6분~8분 소요

In [6]:
pwd

'C:\\Users\\smell\\마케팅 텍스트 마이닝'

In [18]:
# query = "SK매직 가스레인지" 부분에서 원하는 검색어만 변경.

no = 0                 # 몇개의 포스트를 저장하였는지 세기 위한 index
query = "sk매직 전자레인지"   # 검색을 원하는 문자열로서 UTF-8로 인코딩한다. 
                             # 여기를 내가 원하는 검색어 
display = 10           # 검색 결과 출력 건수 지정, 10(기본값),100(최대)
start = 1              # 검색 시작 위치로 최대 2000까지 가능
sort = "sim"           # 정렬 옵션: sim(유사도순, 기본값), date(날짜순)

# 블로그를 DataFrame에 저장
df = pd.DataFrame(columns=("Title", "Link", "Description", "Blogger Name", "Blogger Link", "Post Date", "Post Contents"))

blog_count = get_blog_count(query, display)
for start_index in range(start, blog_count + 1, display):
    get_blog_post(query, display, start_index, sort)

Last build date: Thu, 14 Oct 2021 17:32:55 +0900
Total: 5618
Start: 1
Display: 10
Blog total: 562
Blog count: 562
##########################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################

## 1_6. 수집된 데이터 파일로 저장한 후 불러오기

### ** 코드파일들을 저장한 폴더 내에 data라는 이름의 하위 폴더를 미리 생성하여 향후 여러 데이터 파일들을 저장

In [19]:
df.shape

(570, 7)

In [20]:
df.to_csv("C:/Users/smell/OneDrive/바탕 화면/tool 모음/수업/2학기 수업/파이썬 텍스트 마이닝/텍스트 파이썬/SK_Microwave.csv", header=True, index=False, encoding='utf-8-sig')

In [21]:
writer = pd.ExcelWriter("C:/Users/smell/OneDrive/바탕 화면/tool 모음/수업/2학기 수업/파이썬 텍스트 마이닝/텍스트 파이썬/SK_Microwave.xlsx", engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1') # xslx 파일로 저장 
writer.close() # Pandas writer 객체 닫기

In [23]:
len(df) 

570

In [24]:
df = pd.read_csv("SK_Microwave.csv", encoding='utf-8-sig') # 디렉토리 올려놓음 코드 너무 길어서  

In [26]:
df.head() 

,Title,Link,Description,Blogger Name,Blogger Link,Post Date,Post Contents
0,[내돈내산] SK매직 전자레인지 (MWO-20EC2) 구매 후기,https://blog.naver.com/1320god?Redirect=Log&lo...,전자레인지 전체적으로 무난할 거라고 생각.. SK매직이면 브랜드도 좋다고 생각.. ...,무늬를 찾아서,https://blog.naver.com/1320god,20210102,\n\n\n\n\n\n\n제품\n\n\n\n\n[내돈내산] SK매직 전자레인지 (M...
1,[생활가전리뷰] SK매직 전자레인지 복합오븐 스팀청소도...,https://blog.naver.com/ahj518?Redirect=Log&log...,합니다ㅎㅎ SK매직 전자레인지 겸 복합오븐 기능으로는 ! 오븐 / 구이 / 홈베이킹...,라라린 예비엄마 성장기,https://blog.naver.com/ahj518,20200626,\n\n\n\n\n\n\n제품리뷰\n\n\n\n\n[생활가전리뷰] SK매직 전자레인...
2,SK매직 전자레인지 출력 조리시간 설정 팝콘 사용법 알아봐요,https://blog.naver.com/ujin0917?Redirect=Log&l...,그래서 자주 사용하지 않아도 가끔은 간절히 필요해 지는 SK매직 전자레인지 사용법에...,쩡성이의 각종 가전 레시피,https://blog.naver.com/ujin0917,20211001,\n\n\n\n\n\n\n\n\n\n가전제품\n\n\n\n\nSK매직 전자레인지 출...
3,SK매직 전자레인지 MWO-20EC6 내돈내산,https://blog.naver.com/bichon-haru?Redirect=Lo...,ㅋㅋㅋㅋㅋ 물론 같은 SK 매직 브랜드라 그런 늬낌이겠지만 뭔가 느낌이 넘나 비슷한...,비숑프리제 하루와 나무,https://blog.naver.com/bichon-haru,20200531,\n\n\n\n\n\n\nㄴ엄빠꺼\n\n\n\n\nSK매직 전자레인지 MWO-20E...
4,SK매직 고급형 전자레인지 20L 사용해본 후기,https://blog.naver.com/sakurasake2?Redirect=Lo...,SK매직 고급형 전자레인지 20L 사용 후기를 모아보았습니다. 구매하려고 하셨다면 ...,요나의 일상,https://blog.naver.com/sakurasake2,20210901,\n\n\n\n\n\n\n생활용품\n\n\n\n\nSK매직 고급형 전자레인지 20L...


In [28]:
df.tail()

,Title,Link,Description,Blogger Name,Blogger Link,Post Date,Post Contents
565,"천안 시티자이 SK매직 식기세척기, 오븐시공, 설치, 판매",https://blog.naver.com/built_in09?Redirect=Log...,명가 sk매직의 시그니처 모델로 이름은 EONB401SA 이고 컨벡션 방식의 스팀오...,진영빌트인 빌트인가전 전문업체,https://blog.naver.com/built_in09,20181025,\n\n\n\n\n\n\n\n\n식기세척기\n\n\n\n\n\n\n\n\n\n\n천...
566,[추천후기] SK 매직 20L 버튼식 전자렌지 MWO-20EC7 왕추천~!,https://blog.naver.com/dlaekqlsh3?Redirect=Log...,추천 SK 매직 전자레인지 자취 전자레인지 추천 MWO-20EC7 편지의 감동과 함...,"로미U, 오롯이 나만을 위한",https://blog.naver.com/dlaekqlsh3,20181104,\n\n\n\n\n\n\n\n\n방꾸미기\n\n\n\n\n\n\n\n\n\n\n[추...
567,SK매직 5단계 출력 조절 전자레인지 MWOHM2A1MDBL...,https://blog.naver.com/refurblic-1?Redirect=Lo...,이제 다 아시겠지요^^ 가정에 하나쯤 있는 전자레인지 이지만 한번 구매하면 오랫동안...,.,https://blog.naver.com/refurblic-1,20200124,\n\n\n\n\n\n\n오븐/전자레인지/전기그릴\n\n\n\n\nSK매직 5단계 ...
568,"SK매직 복합오븐, 대기업의 뻥스펙",https://blog.naver.com/shyamoyed_7?Redirect=Lo...,"SK매직한테 뒷통수 맞는 느낌도 들고, 브랜드에 대한 배신감도 들고, 내가 뭔가 오...",22C 오피니언 리더를 위해.,https://blog.naver.com/shyamoyed_7,20200708,"\n\n\n\n\n\n\n낙서장\n\n\n\n\nSK매직 복합오븐, 대기업의 뻥스펙..."
569,복합오븐기/SK매직/EON-C512F/EON-C514F/최저가보다싸게사기...,https://blog.naver.com/fallintomiss?Redirect=L...,며칠전부터 계속 검색하던 중 LG 디오스 광파오븐 32L랑 SK매직의 오븐기를 보다...,아미쮸의 초콜릿공장 :-D,https://blog.naver.com/fallintomiss,20190228,\n\n\n\n\n\n\n７.Review\n\n\n\n\n복합오븐기/SK매직/EON...


In [8]:
# 수집된 블로그 텍스트 데이터를 분석용 csv 파일로 저장 

df.to_csv("C:/Users/smell/OneDrive/바탕 화면/tool 모음/수업/2학기 수업/파이썬 텍스트 마이닝/텍스트 파이썬/Microwave.csv", header=True, index=False, encoding='utf-8-sig') 


In [ ]:
# 아래는 그냥 전자레인지 했을 때 입니다. SK매직 전자래인지는 570개가 나와서 그냥 999개를 다 크롤링 해보고 싶었습니다. 
# 파일은 SK매직으로 제출했습니다. 

In [9]:
# 데이터를 읽기 위한 xslx 파일로 저장
writer = pd.ExcelWriter("C:/Users/smell/OneDrive/바탕 화면/tool 모음/수업/2학기 수업/파이썬 텍스트 마이닝/텍스트 파이썬/Microwave.xlsx", engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1') # xslx 파일로 저장 
writer.close() # Pandas writer 객체 닫기

In [10]:
# 저장된 csv 파일 불러오고 확인하기
df_1 = pd.read_csv("Microwave.csv", encoding='utf-8-sig') # 디렉토리 올려놓음 코드 너무 길어서  

In [12]:
len(df_1)  # 파일이 담고 있는 전체 텍스트 개수 확인

999

In [13]:
df_1.head()  # 상위 5개 텍스트 확인

,Title,Link,Description,Blogger Name,Blogger Link,Post Date,Post Contents
0,전자레인지 청소 수월하게 해결해봄!,https://blog.naver.com/loveyunhi?Redirect=Log&...,천연오일 거품 덕분인건지 전자레인지 내부에서 상큼한 오렌지 향까지 나더라고요. 내부...,예쁜 고구마,https://blog.naver.com/loveyunhi,20210809,\n\n\n\n\n\n\n리뷰 게시판\n\n\n\n\n전자레인지 청소 수월하게 해결...
1,꾸덕한 그릭요거트 만들기 전자레인지 vs 밥솥,https://blog.naver.com/gtbaby35?Redirect=Log&l...,^^ 재료 우유 1L 액티비아 요거트 (농후발효유) 130ml 1개 전자레인지 사용...,삼남매집,https://blog.naver.com/gtbaby35,20210910,\n\n\n\n\n\n\n\n\n\n소꽁요리\n\n\n\n\n꾸덕한 그릭요거트 만들...
2,홈브런치 모닝빵 계란빵 만들기 전자레인지 에어프라이어 요리...,https://blog.naver.com/keroann?Redirect=Log&lo...,요리 전자레인지 요리로도 익혀서 먹어보았는데요 확실히 두 가지가 다름이 있어 취향대...,꿈꾸는 그곳!! 내가 만들어보자!!,https://blog.naver.com/keroann,20211009,\n\n\n\n\n\n\n홈메이드쿡\n\n\n\n\n홈브런치 모닝빵 계란빵 만들기 ...
3,계란찜 만들기 전자레인지 치즈 쭉~ 부드러운 계란요리,https://blog.naver.com/mijinkim80?Redirect=Log...,또 #전자레인지계란찜 아니겠어요? 물론 뚝배기에서 해도 그 나름의 맛이 있고 찜기에...,MJ의후다닥레시피~♥,https://blog.naver.com/mijinkim80,20210908,\n\n\n\n\n\n\n매일 밥상\n\n\n\n\n계란찜 만들기 전자레인지 치즈 ...
4,계란찜 만들기 전자레인지 사용하여 쉽고 맛있게~,https://blog.naver.com/yee1036?Redirect=Log&lo...,계란찜 만들기 전자레인지 사용하여 쉽고 맛있게~ 어제는 심심해서 몸이 배배 꼬인다는...,똥썹모친의 맛이야기,https://blog.naver.com/yee1036,20210805,\n\n\n\n\n\n\n요리 쉽죠잉\n\n\n\n\n계란찜 만들기 전자레인지 사용...


In [18]:
df_1.tail()   # 마지막 5개 텍스트 확인

,Title,Link,Description,Blogger Name,Blogger Link,Post Date,Post Contents
994,+전자레인지 청소법 식초와 귤껍질로 냄새제거도 완벽해~,https://blog.naver.com/skckzo83?Redirect=Log&l...,"전자레인지 청소법 식초, 귤 전자레인지 냄새제거 영상으로도 준비했어요~ 구독과 좋아...",안녕 뉴흔둥이네,https://blog.naver.com/skckzo83,20200210,\n\n\n\n\n\n\n취향이 담긴 살림\n\n\n\n\n+전자레인지 청소법 식초...
995,전자레인지 계란찜 만들기,https://blog.naver.com/hee81hoo?Redirect=Log&l...,전자레인지 계란찜 만들기 호다닥 만들기 좋은 계란찜. 특별한 반찬 없을 때 종종 만...,은야쟁이의 오렌지 블랙박스,https://blog.naver.com/hee81hoo,20200615,\n\n\n\n\n\n\n반찬\n\n\n\n\n전자레인지 계란찜 만들기 \n\n\n...
996,비락 멸균우유 전자레인지 돌려서 마셔요,https://blog.naver.com/midary1009?Redirect=Log...,그리고 전자레인지로 사용 시 환경호르몬 배출이 없는 친화경포장재에요 입구만 컷해서 ...,말랑말랑 하루일기,https://blog.naver.com/midary1009,20210121,\n\n\n\n\n\n\n요것저것\n\n\n\n\n비락 멸균우유 전자레인지 돌려서 ...
997,전자레인지 추천 월풀 디자인까지 산뜻하다,https://blog.naver.com/loveme90000?Redirect=Lo...,다양한 기능들이 있는 화이트 컬러의 월풀 전자레인지 추천 해요 결혼 할때 구입했던 ...,대구 골드스푼의 사적인공간!,https://blog.naver.com/loveme90000,20210224,\n\n\n\n\n\n\n생활용품리뷰\n\n\n\n\n전자레인지 추천 월풀 디자인까...
998,전자레인지 요리 노오븐컵빵 4분이면 완성,https://blog.naver.com/dhdmsdo79?Redirect=Log&...,종이컵에 버터를 조금씩 넣은 후 전자레인지에 20초 정도 돌려 녹여준 후 붓을 이용...,제시의 달빛부엌,https://blog.naver.com/dhdmsdo79,20200528,\n\n\n\n\n\n\n후룩 면\n\n\n\n\n전자레인지 요리 노오븐컵빵 4분이...


# 2. selenium을 활용한 네이버 쇼핑후기 수집하기 

### *** chromedriver가 pc에 없는 경우 미리 다운 받아서 코드가 있는 동일한 폴더에 설치할 것 ***

   1. 크롬드라이버 설치하기 전에 본인PC의 크롬 브라우저 버전 확인하기
     ... 크롬 브라우저 우측 상단의 크롬맞춤설정 및 제어 클릭 
         --> 좌측 중간에 chrome 정보 
         --> 본인의 크롬정보 확인 (예: 93.0.4577.82)
     
    2.아래 사이트에서 본인의 크롬버전에 맞는 크롬 드라이버를 다운받아 압출파일을 풀고 바탕화면에 설치하기   
	https://sites.google.com/a/chromium.org/chromedriver/downloads


    3. 본인의 파이썬 코드와 같은 폴더(보기: PTM2021)로 옮겨 놓기.
    
    

## 2_1. 필요한 라이브러리 설치

In [19]:
from selenium.webdriver import Chrome # Prompt 가서  pip install selenium이거 설치해야 된다!!!!!!! 
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
from time import sleep
from urllib.parse import quote_plus
import requests
import re
import pandas as pd
import numpy as np
import os

## 2_2. 쇼핑후기 텍스트 데이터를 담는 데이터 프레임 생성 

In [21]:
def add_dataframe(name, category,date, title, reviews, stars, cnt):  # 크롤링한 데이터 데이터 프레임 생성하는 함수 
    
    df1=pd.DataFrame(columns=['type', 'category', 'date', 'title', 'review', 'star'])
    n=1
    if (cnt>0):
        for i in range(0,cnt-1):
            df1.loc[n]=[name, category, date[i], titles[i], reviews[i], stars[i]] # n 행에 접근한다. 
            i+=1
            n+=1
    else:
        df1.loc[n]=[name, category, 'null' , 'null', 'null', 'null']
        n+=1    
    return df1

## 2_2. 검색할 키워드 입력  

- 다음의 주소를 클릭하여 네이버 쇼핑몰에 직접 접속한 후 핵심어(브랜드 제품명)를 입력하여 검색

 https://shopping.naver.com/   



## 2_3. 검색할 페이지 찾아서 쇼핑후기 텍스트 수집하기



1) 네이버 쇼핑몰 페이지에서 검색할 검색어 입력 (보기: sk매직 가스레인지)  

2) 검색된 상품 리스트를 '리뷰 많은 순'으로 정렬

3) 우측에 다수의 쇼핑몰이 '쇼핑몰별 최저가'로 제시되어 있는 모델들 선정 (쇼핑몰이 하나만 제시된 모델은 해당 쇼핑몰로 이동)

4) 복수의 쇼핑몰이 제시된 모델 중 리뷰가 1,000개 남짓한 제품 선정 (리뷰가 너무 많으면 별점이 편중되어 수집) 

5) 최종 선택된 모델이 있는 사이트에서 쇼핌몰 리뷰 섹션을 클릭

6) 리뷰 섹션의 주소(url) 복사하여 ns_address = "..."에 붙이기

7) 별도의 창이 열려 쇼핑후기 사이트가 자동으로 구동되다가 없어지는 동안 스크롤 건드리지 말고 기다릴 것. 


In [23]:
# 아래 코드 중에서 수정할 부분
# 1) name=['sk매직 가스레인지'] 에서 검색어 수정
# 2) ns_address = "...." 부분에 수집할 쇼핑몰 리뷰 페이지 url 복사해서 붙여넣기
# 3) 크롬드라이버 경로 수정


name=['SK매직 MWO-20MC9']   # 검색할 브랜드 및 제품명 이름 넣기(**) # 전자레인지 제품명을 적음 
category=['별점']


# 수집할 '쇼핑몰리뷰' 페이지 url 복사해서 붙여넣기 (**)
ns_address = "https://search.shopping.naver.com/catalog/11519043822?query=Sk%EC%A0%84%EC%9E%90%EB%A0%88%EC%9D%B8%EC%A7%80&NaPm=ct%3Dkulnkvnc%7Cci%3D3764ed0c327b51df8e944ba2e28fb8d915523df2%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3Ddf2fafab20fbc63c71cde3a1b0b1a96b4c5b7885"
 
shoppingmall_review_css="#snb > ul > li:nth-child(2) > a"

header = {'User-Agent': ''}
d = webdriver.Chrome('chromedriver.exe') # chromedriver.exe가 있는 경로를 입력해주세요. (**)
d.implicitly_wait(3)
d.get(ns_address)
req = requests.get(ns_address,verify=False)
html = req.text 
soup = BeautifulSoup(html, "html.parser")
sleep(2)

#쇼핑몰 리뷰 보기 & # 리뷰 가져오기
element = d.find_element_by_css_selector(shoppingmall_review_css).click() #스크롤 건드리면 안됨
sleep(2)

name_=name[0]
category_=category[0]
dates, reviews, titles, stars = [], [], [], []
cnt=1
page=1

while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)

    while True: # 한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생 # 20페이지씩 50개 즉 1000개? 
        try:
            star = d.find_element_by_css_selector("#section_review > ul > li:nth-child("+str(j)+") > div.reviewItems_etc_area__2P8i3 > span.reviewItems_average__16Ya-").text
            stars.append(star)
            date = d.find_element_by_css_selector("#section_review > ul > li:nth-child("+str(j)+") > div.reviewItems_etc_area__2P8i3 > span:nth-child(4)").text
            dates.append(date)
            title = d.find_element_by_css_selector("#section_review > ul > li:nth-child("+str(j)+")> div.reviewItems_review__1eF8A > div > em").text
            titles.append(title)
            review = d.find_element_by_css_selector("#section_review > ul > li:nth-child("+str(j)+") > div.reviewItems_review__1eF8A> div > p.reviewItems_text__XIsTc").text
            reviews.append(review)
            j+=1
            print(cnt, star, date, title, "\n") # 리뷰index, 별점, 제목 출력
            cnt+=1
        except:  
            break # 한페이지에 20개 리뷰 마지막 리뷰에서 error 발생 후, 종료
            
    sleep(2)

    if page < 11: # 11 page까지 크롤링 됨 다음으로 넘어가야하기 떄문에 ! 숫자 변경하지 마세요! 
        try: # 리뷰의 마지막 페이지에서 error발생
            page +=1
            next_page=d.find_element_by_css_selector("#section_review > div.pagination_pagination__2M9a4 > a:nth-child("+str(page)+")").click()
        except: 
            break # 리뷰의 마지막 페이지에서 process 종료 

    else:
        try: # page 11 부터 
            page+=1
            if page%10==0:
                next_page=d.find_element_by_css_selector('#section_review > div.pagination_pagination__2M9a4 > a.pagination_next__3ycRH').click()
                
            if page == 51: # 크롤링 원하는 페이지 +1 적용해서 변경하세요 (**)
                break
        except:
            break

d.close() # 크롤링이 끝나면 자동으로 창이 닫힘
          # 함수가 1000개 까지 여서 1000개넘어도 1000개 까지(?)

C:\Users\smell\anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search.shopping.naver.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


페이지 1 

1 평점5 20.03.15. 잘 받았습니다~!배송은 빠르지 않았지만 전자제품은 시간이 좀 걸리니~ 많이 늦지는 않았습니다~그런데 배송 받고 보니...사진처럼 부딪히고 찢어진 것들이 보여서~!안의 제품에 문제는 

2 평점5 21.01.22. 제품자체는 디자인 기능 가격 모두 마음에 듭니다만.... 오븐에어프라이 기능과합쳐진 멀티제품으로 생각이 바뀌게되어... 반품 하려고했으니 일주일이 지나버렸네요..지인한테 넘기던지 

3 평점3 20.08.09. 포장 잘해서 적당히 도착 했는데저가 제풍이라고 불량제품-(표시한부분과 그위에 일자로 길게, 긴 부분은 사진으론 안보여 촬영 포기)녹이난거 같은데 그냥 동장해서 조립 판매...이건 

4 평점5 20.10.18. 장점: 일단 간편하고 간단하게 사용하기 좋습니다.... 

5 평점5 20.09.13. 고장나서샀는데 크기가 아담합니다^^;;물론 리터계산을 안한것은아닌데 지금까지바꾸기전에쓰던 전자렌지가 무척 컸었나봅니다.사용은잘되구요, 다만크기만 상관없다면 소형가전추천하고싶습니다. 

6 평점5 21.01.01. 6만원초중반대에 구매했는데 가격이 올랐네요 ! 잘산거 같아요첨에 작동이 안되는줄 알았는데,,, 나이드신 분이 사용하시는거라사용법을 잘몰랐다고 연락와가지고 불량 아닌거 확인하고 안심 

7 평점1 20.12.13. 이쁜 쓰레기 입니다. 

8 평점5 20.12.02. 필히 

9 평점5 19.08.08. 십수년된 삼성 전저렌지 드뎌 사망.급하게 검색해... 

10 평점5 19.07.26. 잘 받았습니다 ~~
조카녀석이 혼자 독립을 하게 되어 

11 평점4 19.10.06. 디자인.배송.가격 모두 만족합니다..
그런데 기존에 액 

12 평점5 19.08.04. 저렴하게 잘 산거 같네요 .배송도 생각보다 매우 빨랐구 

13 평점1 18.12.23. 최악이에요 

14 평점5 19.01.04. 최고예요 

15 평점5 19.08.10. 롯데택배는 어디서나 문제네요. 전화도 없고, 고객이 받았다고 맘대

## 2_4. 수집된 데이터 저장하기

In [24]:
shop_df = add_dataframe(name_,category_,dates, title, reviews, stars, cnt)
shop_df

,type,category,date,title,review,star
1,SK매직 MWO-20MC9,별점,20.03.15.,잘 받았습니다~!배송은 빠르지 않았지만 전자제품은 시간이 좀 걸리니~ 많이 늦지는 ...,잘 받았습니다~!\n배송은 빠르지 않았지만 전자제품은 시간이 좀 걸리니~ 많이 늦지...,평점5
2,SK매직 MWO-20MC9,별점,21.01.22.,제품자체는 디자인 기능 가격 모두 마음에 듭니다만.... 오븐에어프라이 기능과합쳐진...,제품자체는 디자인 기능 가격 모두 마음에 듭니다만.... 오븐에어프라이 기능과합쳐진...,평점5
3,SK매직 MWO-20MC9,별점,20.08.09.,포장 잘해서 적당히 도착 했는데저가 제풍이라고 불량제품-(표시한부분과 그위에 일자로...,포장 잘해서 적당히 도착 했는데\n저가 제풍이라고 불량제품-(표시한부분과 그위에 일...,평점3
4,SK매직 MWO-20MC9,별점,20.10.18.,장점: 일단 간편하고 간단하게 사용하기 좋습니다....,장점: 일단 간편하고 간단하게 사용하기 좋습니다.\n\n디자인이 깔끔합니디ㅣ.\n\...,평점5
5,SK매직 MWO-20MC9,별점,20.09.13.,고장나서샀는데 크기가 아담합니다^^;;물론 리터계산을 안한것은아닌데 지금까지바꾸기전...,고장나서샀는데 크기가 아담합니다^^;;\n물론 리터계산을 안한것은아닌데 지금까지바꾸...,평점5
...,...,...,...,...,...,...
996,SK매직 MWO-20MC9,별점,20.08.27.,안전하게 잘받았습니다~~,안전하게 잘받았습니다~~,평점5
997,SK매직 MWO-20MC9,별점,20.08.30.,가성비 좋은 제품입니다,가성비 좋은 제품입니다,평점5
998,SK매직 MWO-20MC9,별점,20.08.30.,쓰기좋아요 만족합니다,쓰기좋아요 만족합니다,평점5
999,SK매직 MWO-20MC9,별점,20.08.22.,잘받았어요 번창하세요,잘받았어요 번창하세요,평점3


In [25]:
shop_df['date'] = shop_df['date'].str.replace(".","")

year ="20"

shop_df['date'] = year + shop_df['date']
shop_df['date']

1       20200315
2       20210122
3       20200809
4       20201018
5       20200913
          ...   
996     20200827
997     20200830
998     20200830
999     20200822
1000    20200822
Name: date, Length: 1000, dtype: object

크롤링된 파일 저장하기

In [26]:
shop_df

,type,category,date,title,review,star
1,SK매직 MWO-20MC9,별점,20200315,잘 받았습니다~!배송은 빠르지 않았지만 전자제품은 시간이 좀 걸리니~ 많이 늦지는 ...,잘 받았습니다~!\n배송은 빠르지 않았지만 전자제품은 시간이 좀 걸리니~ 많이 늦지...,평점5
2,SK매직 MWO-20MC9,별점,20210122,제품자체는 디자인 기능 가격 모두 마음에 듭니다만.... 오븐에어프라이 기능과합쳐진...,제품자체는 디자인 기능 가격 모두 마음에 듭니다만.... 오븐에어프라이 기능과합쳐진...,평점5
3,SK매직 MWO-20MC9,별점,20200809,포장 잘해서 적당히 도착 했는데저가 제풍이라고 불량제품-(표시한부분과 그위에 일자로...,포장 잘해서 적당히 도착 했는데\n저가 제풍이라고 불량제품-(표시한부분과 그위에 일...,평점3
4,SK매직 MWO-20MC9,별점,20201018,장점: 일단 간편하고 간단하게 사용하기 좋습니다....,장점: 일단 간편하고 간단하게 사용하기 좋습니다.\n\n디자인이 깔끔합니디ㅣ.\n\...,평점5
5,SK매직 MWO-20MC9,별점,20200913,고장나서샀는데 크기가 아담합니다^^;;물론 리터계산을 안한것은아닌데 지금까지바꾸기전...,고장나서샀는데 크기가 아담합니다^^;;\n물론 리터계산을 안한것은아닌데 지금까지바꾸...,평점5
...,...,...,...,...,...,...
996,SK매직 MWO-20MC9,별점,20200827,안전하게 잘받았습니다~~,안전하게 잘받았습니다~~,평점5
997,SK매직 MWO-20MC9,별점,20200830,가성비 좋은 제품입니다,가성비 좋은 제품입니다,평점5
998,SK매직 MWO-20MC9,별점,20200830,쓰기좋아요 만족합니다,쓰기좋아요 만족합니다,평점5
999,SK매직 MWO-20MC9,별점,20200822,잘받았어요 번창하세요,잘받았어요 번창하세요,평점3


In [27]:
# 수집된 블로그 텍스트 데이터를 분석용 csv 파일로 저장 

shop_df.to_csv("Microwave_S.csv", header=True, index=False, encoding='utf-8-sig') 

xslx 파일로 저장허기

In [28]:
shop_crawling_xlsx = pd.ExcelWriter('Microwave_S.xlsx', engine='xlsxwriter')  

shop_df.to_excel(shop_crawling_xlsx, sheet_name='Sheet1') 

shop_crawling_xlsx.close()

## 2_5. 저장한 텍스트 데이터 불러오기

In [29]:
import pandas as pd # 파일경로를 jupter notebook에 넣었다면 read.csv에 전체 코드 입력은 불필요 

df_2 = pd.read_csv('Microwave_S.csv', encoding='utf-8-sig') # csv 파일 불러오기

In [30]:
len(df_2)

1000

In [31]:
df_2.head()

,type,category,date,title,review,star
0,SK매직 MWO-20MC9,별점,20200315,잘 받았습니다~!배송은 빠르지 않았지만 전자제품은 시간이 좀 걸리니~ 많이 늦지는 ...,잘 받았습니다~!\n배송은 빠르지 않았지만 전자제품은 시간이 좀 걸리니~ 많이 늦지...,평점5
1,SK매직 MWO-20MC9,별점,20210122,제품자체는 디자인 기능 가격 모두 마음에 듭니다만.... 오븐에어프라이 기능과합쳐진...,제품자체는 디자인 기능 가격 모두 마음에 듭니다만.... 오븐에어프라이 기능과합쳐진...,평점5
2,SK매직 MWO-20MC9,별점,20200809,포장 잘해서 적당히 도착 했는데저가 제풍이라고 불량제품-(표시한부분과 그위에 일자로...,포장 잘해서 적당히 도착 했는데\n저가 제풍이라고 불량제품-(표시한부분과 그위에 일...,평점3
3,SK매직 MWO-20MC9,별점,20201018,장점: 일단 간편하고 간단하게 사용하기 좋습니다....,장점: 일단 간편하고 간단하게 사용하기 좋습니다.\n\n디자인이 깔끔합니디ㅣ.\n\...,평점5
4,SK매직 MWO-20MC9,별점,20200913,고장나서샀는데 크기가 아담합니다^^;;물론 리터계산을 안한것은아닌데 지금까지바꾸기전...,고장나서샀는데 크기가 아담합니다^^;;\n물론 리터계산을 안한것은아닌데 지금까지바꾸...,평점5


In [32]:
df_2.tail() # 뒤에서 0~4번째 행 출력
            # 평점이 몇인지? 

,type,category,date,title,review,star
995,SK매직 MWO-20MC9,별점,20200827,안전하게 잘받았습니다~~,안전하게 잘받았습니다~~,평점5
996,SK매직 MWO-20MC9,별점,20200830,가성비 좋은 제품입니다,가성비 좋은 제품입니다,평점5
997,SK매직 MWO-20MC9,별점,20200830,쓰기좋아요 만족합니다,쓰기좋아요 만족합니다,평점5
998,SK매직 MWO-20MC9,별점,20200822,잘받았어요 번창하세요,잘받았어요 번창하세요,평점3
999,SK매직 MWO-20MC9,별점,20200822,아주간편하고조아요옹,아주간편하고조아요옹,평점5
